### Introduction

You might have already tried to calculate the power spectrum using pacakges like `nbodykit`. But do you know what actually happened when you were running codes like `nbodykit.algorithms.fftpower.FFTPower(...)`? If not, no worries! Since that is exactly what you will be learning from this notebook.

### Goal:
* Understand Fourier Transformations
* Learn to calculate Power Spectrum
* Learn to calculate Potential and Force-field
* Learn how to get from linear field to Zel'dovich

In [1]:
%matplotlib inline

import numpy as np 
import matplotlib.pyplot as plt

import sys
sys.path.append("../../")

import sheet_unfolding.sim as sim

from matplotlib.colors import LogNorm

## Calculate the Power Spectrum

The definition for power spectrum is really simple
\begin{align}
  P(k) = \langle |\delta(|\vec{k}|)|^2 \rangle,
\end{align}
where $\delta(\vec{k})$ is the overdensity field in Fourier space, i.e. the Fourier transform of $\delta(\vec{x})$. In simulations, $\delta(\vec{x})$ is usually obtained based on some grids, every grid cell corresponds to a coordinate $\vec{x_i}$ and the overdensity at the coordinate is $\delta(\vec{x_i})$. Similarly, when you transformed it into Fourier space, every grid cell corresponds to a wavenumber $\vec{k_i}$ and the overdensity at the wavenumber is $\delta(\vec{k_i})$. Therefore, to get $P(k)$, you need to 
* Get the $\delta(\vec{x})$ at $a=1$ using `sim.ic.IC2DCosmo.get_delta()`,
* Fourier transform $\delta(\vec{x})$ and get $\delta(\vec{k})$ using `np.fft.fft2()`,
* Fourier transform the coordinate of the grids $\vec{x}$ to get $\vec{k}$ (this can be tricky, a code is provided in `get_kmesh()`),
* Get the the squared modulus of $\delta(\vec{k})$ and the modulus of $\vec{k}$ at every grid cells.
* Make proper bins for $k$, and calculate the average value of $|\delta(|\vec{k}|)|^2$ in each k bin as the $P(k)$ in the bin.
* Plot your results.
* Compare your results with the one derived from `sim.ic.power_eisenstein_smoothed_2d`. Does they overlap with each other? Try different smoothing radius for the initial condition setting, what does the smoothing do to the power spectrum? Can you explain why it behaves in this way?

**Normalisations**

1. Notice that the discrete Fourier transform (DFT) uses a different normalisation. To convert them you need
\begin{align}
  \delta_k = \delta_{k, DFT} \frac{\sqrt{L}^d}{N^d}
\end{align}
where $d = 2$ is the dimension, $L$ is the side length of your simulation box, and $N$ is the number of grids on each side.

2. To compare your power spectrum with the one from `sim.ic.power_eisenstein_smoothed_2d`, the latter need to be converted into the same units by multiplying the return of `sim.ic.power_eisenstein_smoothed_2d` by $f^2_{\mathrm{norm}}$, where $f_{\mathrm{norm}}=$`0.8/sim.ic.get_sigmaR(8.)`.

You can also try to do the same calculation for the dimensionless power spectrum

\begin{align}
  D(k) = P(k) k^2 / (2 \pi).
\end{align}

(Note that this relation (2D) is different in 3 dimensions).

In [ ]:
def get_kmesh(npix, L, real=False):
    """ construct a uniform mesh of kvectors (for usage with np.fft.fftn)

    Parameters:
    ----------
    npix : sequence of integer
        Number of pixels in each dimension
    L :  float
        Size of the domain (in real space) - typically the boxsize
    real : bool
        Set to true for usage with np.fft.rfftn

    Returns:
    ----------
    knd : (npix[0], npix[1], ..., len(npix)) array of k-vectors

    Example:
    ----------
    k = get_kmesh((128,128,128), 10., real=False)
    => (128,128,128,3) array
    """

    ndim = len(npix)

    L = np.ones_like(npix) * L

    k1d = []
    for i in range(0, ndim):
        if (i == ndim-1) & real:  # last dim can have different shape in real fft
            k1d.append(np.fft.fftfreq(npix[i])[0:(npix[i]//2)+1] * (2*np.pi * npix[i]) / L[i])
        else:
            k1d.append(np.fft.fftfreq(npix[i]) * (2*np.pi * npix[i]) / L[i])

    knd  = np.array(np.meshgrid(*k1d, indexing='ij')).astype("f4")
    knd  = np.rollaxis(knd, 0, ndim+1)  # bring into shape [npix,npix,...,ndim]

    return knd

In [ ]:
L = 400.

bins = np.linspace(0., 512*np.pi/L,100)
ki = 0.5*(bins[1:] + bins[:-1])

fig, axs = plt.subplots(2,1, figsize=(4,7))

for rs in (0.1, 0.5,1., 3.):
    myic = sim.ic.IC2DCosmo(512, L=L, rs=rs, Omega_m=1.)
    delta = myic.get_delta(a=1.)
    
    deltak =  np.fft.fft2(delta) / np.prod(delta.shape) * L
    
    kvec = get_kmesh((512,512), L)
    kabs = np.sqrt(np.sum(kvec**2, axis=-1))
    
    # Fill in code to calculate and plot power spectrum here: 
    # You'll need to average modes in each k-bin <---
    
    
    
    # --->
    
norm = 0.8/sim.ic.get_sigmaR(8.)
axs[0].loglog(ki, norm**2*sim.ic.power_eisenstein_smoothed_2d(ki, dimless=False), ls="dashed", color="black", label="linear spectrum")
axs[1].loglog(ki, norm**2*sim.ic.power_eisenstein_smoothed_2d(ki, dimless=True), ls="dashed", color="black", label="linear spectrum")

axs[0].set_ylim(1e-1,1e3)
axs[1].set_ylim(1e-3,1e4)

for ax in axs:
    ax.grid()
    ax.legend()
    ax.set_xlim(5e-2, None)
axs[1].set_xlabel(r"$k$ [$h$ Mpc$^{-1}$]")
axs[0].set_ylabel(r"$P(k)$ [Mpc$^{2} h^{-2}$]")
axs[1].set_ylabel(r"$\Delta^2 (k)$")

### Target Output
![title](solutions/img/power_spectrum.png)

## Calculate the Potential

Now you should have mastered how to get the power spectrum given real-space overdensity using Fourier transform. There are many other pros of doing things in Fourier space, one of which is that differenciation in real space becomes simple multiplication in Fourier space. In this subsection, we will explore how to calculate gravity in $N$-body simulations quickly by calculating the Poisson's equation in Fourier space.

Poisson's equation and the acceleration equation in real space are
\begin{align}
   \nabla^2 \phi = 4 \pi G\bar{\rho} \delta \\
            \vec{a} = - \nabla \phi.
\end{align}
In Fourier Space they become
\begin{align}
   \phi_k = - \frac{4 \pi G \bar{\rho}\delta_k}{k^2} \\
            \vec{a_k} = - i \vec{k} \phi_k
\end{align}

that means I can calculate $\phi$ and $a$ in real space by doing:
\begin{align}
   \phi = -4 \pi G\bar{\rho} \cdot \rm{IFFT}\left(\frac{\rm{FFT}(\delta)}{k^2}\right)\\
   \vec{a} = 4 \pi G \bar{\rho}\cdot\rm{IFFT}\left(  \frac{i \vec{k}\cdot\rm{FFT}(\delta)}{k^2}\right)
\end{align}

We can also calculate the acceleration (along the x direction for example) from the finite diffenciation:
$\begin{align}
   a_x \approx \frac{\phi(x+dx,y) - \phi(x-dx,y)}{2 dx}
\end{align}$


### Task:
* Calculate Potential Field in real space
* Calculate Acceleration Field (just the x-component) in real space
    - through Fourier Space
    - through finite differences on $\phi$

### Some Notes:
Just use $r_s = 0.1$ for this task. We will set all the constants to 1 for simplicity. You can set phi_k[0,0] explicitly to 0 to avoid problems with zero divisions. IFFT will return complex numbers with complex part = 0, but you can just cast them to real ones with np.real. Because of the numpy broadcasting rules (https://numpy.org/doc/stable/user/basics.broadcasting.html) you may have to do something like this to multiply a field by a vector field: phik[...,np.newaxis] * kvec

In [1]:
# your codes


### Target Output
![title](./solutions/img/force_calculation.png)

## Zel'dovich approximation

Now you should also be able to implement the Zel'dovich approximation yourself (given the linear overdensity) with all the skills you mastered in this notebook. 

You can have a look at the the source codes of `sim.ic.IC2DCosmo.get_x()` for an example of this.